In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as rd
import random
import yfinance as yf
from fredapi import Fred

class Trade:
    def __init__(self, asset, trade_type, price, quantity, status):
        self.asset = asset
        self.trade_type = trade_type
        self.price = price
        self.quantity = quantity
        self.status = status

    def to_dict(self):
        return {
            "asset": self.asset,
            "type": self.trade_type,
            "price": self.price,
            "quantity": self.quantity,
            "status": self.status
        }

class Portfolio:
    def __init__(self, initial_cash, initial_positions):
        self.cash = initial_cash
        self.positions = initial_positions
        
    def buy(self, asset, price, quantity):
        self.cash -= quantity * price
        if asset in self.positions:
           self.positions[asset]["quantity"] += quantity
           self.positions[asset]["price"] = price
        else:
            self.positions[asset] = {
                "quantity": quantity,
                "price": price
            }
        if self.positions[asset]["quantity"] == 0:
            del self.positions[asset]
    
    def sell(self, asset, price, quantity):
        self.cash += quantity * price
        if asset in self.positions:
           self.positions[asset]["quantity"] -= quantity
           self.positions[asset]["price"] = price
        else:
            self.positions[asset] = {
                "quantity": -quantity,
                "price": price
            }
        if self.positions[asset]["quantity"] == 0:
            del self.positions[asset]

class Strategy:
    def generate_signals(self, data):
        raise NotImplementedError("Override this in a subclass")

    def sma(self, data, window):
        """Calculate Simple Moving Average."""
        return data.rolling(window).mean()

    def ema(self, data, window):
        """Calculate Exponential Moving Average."""
        return data.ewm(span=window, adjust=False).mean()

class MovingAverageCrossover(Strategy):
    def __init__(self, short_window=20, long_window=50):
        super().__init__()
        self.short_window = short_window
        self.long_window = long_window

    def generate_signals(self, data):
        data["SMA_short"] = self.sma(data["Close"], self.short_window)
        data["SMA_long"] = self.sma(data["Close"], self.long_window)
        return (data["SMA_short"] > data["SMA_long"]).astype(int)

def random_date(start_date, end_date, max_dist):
    """Generate a random date between start_date and end_date, which are strings (inclusive). max_dist is the maximum possible distance that the
    generated date can have from the start date"""
    # Convert strings to datetime objects
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    if max_dist == None:
        delta_days = (end - start).days
    else:
        delta_days = min((end - start).days, max_dist, 0)
    random_days = random.randint(0, delta_days)
    return start + dt.timedelta(days=random_days)

def download_assets(tickers, start_date=None, end_date=None, interval="1d"):
    """
    Download historical price data for multiple tickers from Yahoo Finance,
    and return a single DataFrame with a 'Ticker' column.
    
    Parameters:
        tickers (list of str): List of ticker symbols, e.g. ["AAPL", "AMZN"]
        start_date (str or None): Start date in "YYYY-MM-DD" format or None
        end_date (str or None): End date in "YYYY-MM-DD" format or None
        interval (str): Data interval, e.g. "1d", "1h", etc.
    
    Returns:
        pd.DataFrame: Combined DataFrame with multi-ticker data
    """
    all_data = []

    for ticker in tickers:
        print(f"Downloading {ticker} data...")
        df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        df = df.reset_index()  # Move Date from index to column
        df["Ticker"] = ticker
        all_data.append(df)

    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

class Backtest:
    def __init__(self, portfolio, strategies, data, start_date, end_date):
        """
        portfolio: Portfolio object
        strategies: dict mapping ticker -> strategy instance
        data: DataFrame with ["Date", "Ticker", "Close", ...]
        start_date: pandas datetime object for start of the simulation
        end_date: pandas datetime object for end of the simulation
        """
        self.portfolio = portfolio
        self.strategies = strategies
        self.data = data.sort_values(["Date", "Ticker"])

    def run(self):
        for date, group in self.data.groupby("Date"):
            for ticker, asset_data in group.groupby("Ticker"):
                if ticker not in self.strategies:
                    continue  # skip tickers without a strategy

                strategy = self.strategies[ticker]
                signal = strategy.generate_signals(asset_data)
                last_signal = signal.iloc[-1]
                price = asset_data["Close"].iloc[-1]

                if last_signal == 1:
                    self.portfolio.buy(ticker, price, quantity=10)
                elif last_signal == -1:
                    self.portfolio.sell(ticker, price, quantity=10)

        return self.portfolio

    def run1(self):
        for date, group in self.data.groupby("Date"):
            for ticker, asset_data in group.groupby("Ticker"):
                if ticker not in self.strategies:
                    continue  # skip tickers without a strategy

                strategy = self.strategies[ticker]
                signal = strategy.generate_signals(asset_data)
                last_signal = signal.iloc[-1]
                price = asset_data["Close"].iloc[-1]

                if last_signal == 1:
                    self.portfolio.buy(ticker, price, quantity=10)
                elif last_signal == -1:
                    self.portfolio.sell(ticker, price, quantity=10)

        return self.portfolio

def backtester_with_dates(portfolio, strategies, data, start_date="2017-01-01", end_date="2025-08-15"):
    '''
    portfolio: Portfolio object
    strategies: dictionary with strategies for each asset
    data: data
    '''

    # Identify errors and problems ("if True" statement used to collapse the code)
    if True:
        # We will append all errors to this list so they can be displayed to the user
        errors = ["List of errors:"]

        # We take care of data-type for portfolio, strategies, data
        if not isinstance(portfolio, Portfolio):
            errors.append("The 'portfolio' object must be an instance of the Portfolio class.")
        if not isinstance(strategies, dict):
            errors.append("The 'strategies' object must be a Python dictionary.")
        if not isinstance(data, pd.DataFrame):
            errors.append("The 'data' object must be a pandas dataframe.")
        
        # We take care of start_date
        if not isinstance(start_date, dateformat):
            errors.append("The 'start_date' object must be a dateformat.")
            status_start_date = "bad"
        else:
            status_start_date = "good"
        
        # We take care of end_date
        if not isinstance(end_date, dateformat):
            errors.append("The 'end_date' object must be a dateformat.")
            status_end_date = "bad"
        else:
            status_end_date = "good"

        # We take care of the relationship between start_date and end_date
        if status_start_date == "good" and status_end_date == "good" and end_date - start_date <= 2:
            errors.append("The 'max_date' must be at least 3 days after the 'min_date'.")
        

    # If any errors are detected, we return the errors instead of running the simulation
    if len(errors) >= 2:
        return errors
    
    # Make sure dates are datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Store length of trading period
    period = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365

    # Run the simulation
    results = Backtest(portfolio, strategies, data, start_date, end_date).run()

    # Calculate the investment return
    roi = results['final cash'] / portfolio.cash - 1
    rounded_roi = 100 * round(roi, 4)

    # Calculate the investment AAGR
    aagr = roi / period
    rounded_aagr = 100 * round(aagr, 4)

    # Calculate the investment AAGR
    cagr = (1 + roi) ** (1 / period) - 1
    rounded_cagr = 100 * round(cagr, 4)

    # Calculate the bank return
    bank_roi = results['final bank cash'] / portfolio.cash - 1
    rounded_bank_roi = 100 * round(bank_roi, 4)

    return {
        "roi": rounded_roi,
        "aagr": rounded_aagr,
        "cagr": rounded_cagr,
        "bank roi": rounded_bank_roi
        
    }







def backtester_random_dates(portfolio, strategies, data, min_date = "2017-01-01", max_date = "2025-08-15", min_period=3, max_period=None, sample_size = 100):

    # Identify errors and problems ("if True" statement used to collapse the code)
    if True:
        # We will append all errors to this list so they can be displayed to the user
        errors = ["List of errors:"]

        # We take care of data-type for portfolio, strategies, data
        if not isinstance(portfolio, Portfolio):
            errors.append("The 'portfolio' object must be an instance of the Portfolio class.")
        if not isinstance(strategies, dict):
            errors.append("The 'strategies' object must be a Python dictionary.")
        if not isinstance(data, pd.DataFrame):
            errors.append("The 'data' object must be a pandas dataframe.")
        
        # We take care of min_date
        if not isinstance(min_date, dateformat):
            errors.append("The 'min_date' object must be a dateformat.")
            status_min_date = "bad"
        else:
            status_min_date = "good"
        
        # We take care of max_date
        if not isinstance(max_date, dateformat):
            errors.append("The 'max_date' object must be a dateformat.")
            status_max_date = "bad"
        else:
            status_max_date = "good"

        # We take care of the relationship between min_date and max_date
        if status_min_date == "good" and status_max_date == "good":
            dates_relative_status = "good"
            if max_date - min_date <=2:
                errors.append("The 'max_date' must be at least 3 days after the 'min_date'.")
        

        # We take care of min_period and its relationship with min_date-to-max_date period.
        if not isinstance(min_period, int):
            errors.append("The 'min_period' object must be an integer.")
            min_period_status = "bad"
        elif min_period <= 2:
            errors.append("The 'min_period' object must be at least 3.")
            min_period_status = "bad"
        elif dates_relative_status == "good" and max_date - min_date < min_period:
            errors.append("The 'min_period' is longer than the time between 'min_date' and 'max_date'.")
        else:
            min_period_status = "good"

        # We take care of max_period
        if  max_period != None:
            if not isinstance(max_period, int):
                errors.append("The 'max_period' object must be an integer or remain empty (no max period).")
            elif max_period <= 2:
                errors.append("The 'max_period' object must be at least 3 or remain empty (no max period).")
            elif min_period_status == "good" and max_period < min_period:
                errors.append("The 'max_period' object must be at least 'min_period' or remain empty (no max period).")
        
        # We take care of sample_size
        if not isinstance(sample_size, int):
            errors.append("The 'sample_size' object must be an integer or remain empty (will resort to a default value of 100).")
        elif sample_size <= 0:
            errors.append("The 'sample_size' object must be at least 1 or remain empty (will resort to a default value of 100).")

    # If any errors are detected we return the errors instead of running the simulations
    if len(errors) >= 2:
        return errors


    # Start parameters for later performance evaluation
    roi_s = []
    aagr_s = []
    cagr_s = []
    periods = []
    over_zero_counter = 0
    over_bank_counter = 0

    # Run simulations
    for i in range(sample_size):
        # Generate random start and end dates for this simulation
        start_date = random_date(min_date, max_date, (pd.to_datetime(max_date) - pd.to_datetime(min_date)).days - min_period)
        end_date = random_date(start_date + pd.Timedelta(days=min_period), max_date, max_period)
        
        # Store length of trading period
        period = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365
        periods.append((pd.to_datetime(end_date) - pd.to_datetime(start_date)).days)


        #Only keep 
        # Run a backtest
        results = Backtest(portfolio, strategies, data, start_date, end_date).run()

        # Calculate the investment return
        roi = results['final cash'] / portfolio.cash - 1

        # Calculate the investment AAGR
        aagr = roi / period
        rounded_aagr = 100 * round(aagr, 4)


        # Calculate the investment AAGR
        cagr = (1 + roi) ** (1 / period) - 1
        rounded_cagr = 100 * round(cagr, 4)

        # Calculate the bank return
        bank_roi = results['final bank cash'] / portfolio.cash - 1

        # Store parameters
        roi_s.append(roi)
        if roi > 0:
            over_zero_counter += 1
        if roi > bank_roi:
            over_bank_counter += 1
    # Find average profit
    avg_roi = roi_s.mean()
    return {
        "sample size": sample_size,
        "average roi": avg_roi,
        "over bank counter": over_bank_counter,
        "over zero counter": over_zero_counter
    }



In [ ]:
# Step 0: Choose strategies
# Strategy 1: Moving average crossover, take profit at 10%, stop loss at 5%
assets_strat_1 = {
    "tickers": ["AAPL", "AMZN"],
    "tp": 0.1,
    "sl": 0.05
}

# Strategy 2: RSI reversal 14, take profit at 5%, stop loss at 1%
assets_strat_2 = {
    "tickers": ["BTC"],
    "tp": 0.05,
    "sl": 0.01
}

# Strategy 3: RSI reversal 10, take profit at 1%, stop loss at 1%
assets_strat_3 = {
    "tickers": [],
    "tp": 0.01,
    "sl": 0.01
}


all_assets = list(set(assets_strat_1["tickers"]) | set(assets_strat_2["tickers"]) | set(assets_strat_3["tickers"]))

# Step 2: Establish initial cash, initial positions and create portfolio
initial_cash = 1000
initial_positions = {}
initial_positions2 = {
    "AAPL": {
        "quantity": 10,
        "price": 500.9
    },
    "AMZN": {
        "quantity": -1,
        "price": 398.98
    }
                      
}

portfolio = Portfolio(initial_cash, initial_positions)

# Step 3: Assign strategies to tickers
strategies = {}
for asset in all_assets:
    if asset in assets_strat_1:
        strategies[asset] = MovingAverageCrossover(short_window=10, long_window=50)
    elif asset in assets_strat_2:
        strategies[asset] = RSIReversal(period=14)
    elif asset in assets_strat_3:
        strategies[asset] = RSIReversal(period=10)

# Step 4: Download and create all relevant data
data = 0

In [ ]:
# RUN A SIMULATION FOR CHOSEN DATES
test = backtester_with_dates(portfolio, strategies, data, start_date="2017-01-01", end_date="2025-08-15")

# If errors were raised we display them
if isinstance(test, list):
    for error in test:
        print(error)

# If no errors were raised we display the results
else:
    print(f"We ran a simulation with this strategy, starting with {portfolio.initial_cash}$, and finishing with {portfolio.cash}$.")
    print(f"The profit was {round(100 * test['roi'], 2)}%.")
    if test['roi'] > test['bank roi']:
        print(f'Our profit was larger than the projected risk-free interest of {round(100 * test['bank roi'], 2)}%.')
    else:
        print(f'Our profit was smaller than the projected risk-free interest of {round(100 * test['bank roi'], 2)}%.')

In [ ]:
# RUN SIMULATIONS FOR RANDOM DATES IN A RANGE
test = backtester_random_dates(portfolio, strategies, data, min_date="2017-01-01", max_date="2025-08-15", min_period=3, max_period=None, sample_size = 100)

# If errors were raised we display them
if isinstance(test, list):
    for error in test:
        print(error)

# If no errors were raised we display the results
else:
    print(f"We ran {test['sample size']} simulations with this strategy, starting with {portfolio.initial_cash}$ in cash and the following stocks:")
    for stock in portfolio.initial_positions:
        print(f"{portfolio.initial_positions[stock]['quantity']} shares of {stock} each priced at {portfolio.initial_positions[stock]['price']}.")

    print(f"On average, the profit was {round(100 * test['average roi'], 2)}%.")
    print(f"{round(100 * test['over bank counter'] / test['sample size'], 2)}% of the simulations ended with more profit than an equivalent bank account.")
    print(f"{round(100 * test['over zero counter'] / test['sample size'], 2)}% of the simulations ended with positive profit.")

In [ ]:
# IF WE DID THE RANDOM BACKTESTER USING THE ORIGINAL FUNCTION

def backtester_random_dates(portfolio, strategies, data, min_date = "2017-01-01", max_date = "2025-08-15", min_period=3, max_period=None, sample_size = 100):

    # Identify errors and problems ("if True" statement used to collapse the code)
    if True:
        # We will append all errors to this list so they can be displayed to the user
        errors = ["List of errors:"]

        # We take care of data-type for portfolio, strategies, data
        if not isinstance(portfolio, Portfolio):
            errors.append("The 'portfolio' object must be an instance of the Portfolio class.")
        if not isinstance(strategies, dict):
            errors.append("The 'strategies' object must be a Python dictionary.")
        if not isinstance(data, pd.DataFrame):
            errors.append("The 'data' object must be a pandas dataframe.")
        
        # We take care of min_date
        if not isinstance(min_date, dateformat):
            errors.append("The 'min_date' object must be a dateformat.")
            status_min_date = "bad"
        else:
            status_min_date = "good"
        
        # We take care of max_date
        if not isinstance(max_date, dateformat):
            errors.append("The 'max_date' object must be a dateformat.")
            status_max_date = "bad"
        else:
            status_max_date = "good"

        # We take care of the relationship between min_date and max_date
        if status_min_date == "good" and status_max_date == "good":
            dates_relative_status = "good"
            if max_date - min_date <=2:
                errors.append("The 'max_date' must be at least 3 days after the 'min_date'.")
        

        # We take care of min_period and its relationship with min_date-to-max_date period.
        if not isinstance(min_period, int):
            errors.append("The 'min_period' object must be an integer.")
            min_period_status = "bad"
        elif min_period <= 2:
            errors.append("The 'min_period' object must be at least 3.")
            min_period_status = "bad"
        elif dates_relative_status == "good" and max_date - min_date < min_period:
            errors.append("The 'min_period' is longer than the time between 'min_date' and 'max_date'.")
        else:
            min_period_status = "good"

        # We take care of max_period
        if  max_period != None:
            if not isinstance(max_period, int):
                errors.append("The 'max_period' object must be an integer or remain empty (no max period).")
            elif max_period <= 2:
                errors.append("The 'max_period' object must be at least 3 or remain empty (no max period).")
            elif min_period_status == "good" and max_period < min_period:
                errors.append("The 'max_period' object must be at least 'min_period' or remain empty (no max period).")
        
        # We take care of sample_size
        if not isinstance(sample_size, int):
            errors.append("The 'sample_size' object must be an integer or remain empty (will resort to a default value of 100).")
        elif sample_size <= 0:
            errors.append("The 'sample_size' object must be at least 1 or remain empty (will resort to a default value of 100).")

    # If any errors are detected we return the errors instead of running the simulations
    if len(errors) >= 2:
        return errors


    # Start parameters for later performance evaluation
    roi_s = []
    over_zero_counter = 0
    over_bank_counter = 0

    # Run simulations
    for i in range(sample_size):
        # Generate random start and end dates for this simulation
        start_date = random_date(min_date, max_date, (pd.to_datetime(max_date) - pd.to_datetime(min_date)).days - min_period)
        end_date = random_date(start_date + pd.Timedelta(days=min_period), max_date, max_period)
        

        # Run simple backtester function with acquired random dates
        test = backtester_with_dates(portfolio, strategies, data, start_date=start_date, end_date=end_date)

        # Calculate the investment return
        roi = test['roi'] / portfolio.cash - 1

        # Calculate the bank return
        bank_roi = test['bank roi'] / portfolio.cash - 1

        # Store parameters
        roi_s.append(roi)
        if roi > 0:
            over_zero_counter += 1
        if roi > bank_roi:
            over_bank_counter += 1
    # Find average profit
    avg_roi = roi_s.mean()
    return {
        "sample size": sample_size,
        "average roi": avg_roi,
        "over bank counter": over_bank_counter,
        "over zero counter": over_zero_counter
    }
